<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182253396" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [69]:
from kaggle_secrets import UserSecretsClient
wandb_key_label = "WANDB_KEY"
wandb_key= UserSecretsClient().get_secret(wandb_key_label)

In [70]:
import git
git.Repo.clone_from('https://github.com/Lumin-Lab/BerCuration', '/kaggle/working/scarf')

<git.repo.base.Repo '/kaggle/working/scarf/.git'>

In [19]:
!pip install -r /kaggle/working/scarf/requirements.txt

  Using cached matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires matplotlib<3.9,>=3.2, but you have matplotlib 3.9.0 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [89]:
import os
def save_csv_file(df, path):
    dir_name = os.path.dirname(path)
    if dir_name:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    df.to_csv(path, index=False)

In [107]:
import pandas
df = pd.read_csv("/kaggle/input/ber-stratified-samples/BER_stratified_sample.csv")[:1000]

/tmp/ipykernel_33/2529511084.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/ber-stratified-samples/BER_stratified_sample.csv")[:1000]


In [99]:
n_splits = 2
output_dir = "/kaggle/working/output"
config_dir="/kaggle/working/scarf/configs"
scarf_model_name = "scarf"
mlp_model_name = "mlp"

In [108]:
from sklearn.model_selection import KFold
import os
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(df)):
    save_csv_file(df.iloc[train_index], f"{output_dir}/split_{i+1}/raw_train.csv")
    save_csv_file(df.iloc[test_index], f"{output_dir}/split_{i+1}/raw_test.csv")

In [109]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_train.csv" \
      --output_csv_name "processed_train" \
      --is_train
    """
    os.system(command)
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_test.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

**Train the SCARF Encoder on the Train Dataset**

In [117]:
scarf_batch_size = 32
scarf_epochs = 1
scarf_lr = 3e-5
scarf_emb_dim = 32
scarf_encoder_depth = 3
scarf_corruption_rate=0.3

In [119]:
import os
"""The trained scarf model is saved in 
/kaggle/working/output/split_{split}/scarf.pt if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_scarf.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --train_data_path="{output_dir}/split_{i+1}/processed_train.csv"\
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr=3e-5 \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name="{scarf_model_name}" \
      --corruption_rate={scarf_corruption_rate} \
      --wandb_project_name='SCARF_Project' \
      --wandb_entity='urbancomp' \
      --wandb_key='{wandb_key}'
    """

    os.system(command)

wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240608_165334-1j4dmf2u
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/1j4dmf2u


Epoch [1/1] - Train Loss: 3.460
Model saved at /kaggle/working/output/split_1/scarf.pt


wandb: \ 0.002 MB of 0.019 MB uploaded
wandb: Run history:
wandb: encoder_train/train_loss █▃▃▅▁▇▆▅▇▇▆▅▄▅▅
wandb: 
wandb: Run summary:
wandb: encoder_train/train_loss 3.92272
wandb: 
wandb: 🚀 View run SCARF_Project at: https://wandb.ai/urbancomp/Scarf/runs/1j4dmf2u
wandb: ⭐️ View project at: https://wandb.ai/urbancomp/Scarf
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240608_165334-1j4dmf2u/logs
wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade,

Epoch [1/1] - Train Loss: 3.439
Model saved at /kaggle/working/output/split_2/scarf.pt


wandb: \ 0.002 MB of 0.019 MB uploaded
wandb: Run history:
wandb: encoder_train/train_loss ▆▃▄▂▃▂▆▂▂▅▂█▁▅▁
wandb: 
wandb: Run summary:
wandb: encoder_train/train_loss 3.90129
wandb: 
wandb: 🚀 View run SCARF_Project at: https://wandb.ai/urbancomp/Scarf/runs/tju93w3x
wandb: ⭐️ View project at: https://wandb.ai/urbancomp/Scarf
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240608_165403-tju93w3x/logs


**Obtain the SCARF embeddings for the Small Train dataset, and save the result**

In [120]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/split_{split}/train.npy if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_train.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="train"
    """

    os.system(command)

Model loaded from /kaggle/working/output/split_1/scarf.pt
Model loaded from /kaggle/working/output/split_2/scarf.pt


**Obtain the SCARF embeddings for the Test dataset, and save the result**

In [121]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/split_{split}/test.npy if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_test.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="test"
    """

    os.system(command)

Model loaded from /kaggle/working/output/split_1/scarf.pt
Model loaded from /kaggle/working/output/split_2/scarf.pt


**Load the training and testing embeddings**

In [122]:
from cleanlab import Datalab
import numpy as np
train_emb = np.load(f"{output_dir}/split_1/{train_embedding_name}.npy")

In [123]:
from cleanlab.outlier import OutOfDistribution
ood = OutOfDistribution()
ood_train_feature_scores = ood.fit_score(features=train_emb)
ood_train_feature_scores = ood.score(features=train_emb)
# ood_test_feature_scores = ood.score(features=test_emb)


Fitting OOD estimator based on provided features ...


In [126]:
ood_train_feature_scores

array([0.583817  , 0.5571576 , 0.53148454, 0.12195943, 0.4530235 ,
       0.5697815 , 0.41915447, 0.41873714, 0.30173627, 0.3359073 ,
       0.36623478, 0.3200014 , 0.5039719 , 0.29378122, 0.48308095,
       0.53739303, 0.528277  , 0.343223  , 0.4126202 , 0.38446197,
       0.6674264 , 0.29366848, 0.50010955, 0.6280158 , 0.4387273 ,
       0.44971368, 0.5746294 , 0.24315289, 0.45362303, 0.38513872,
       0.6210135 , 0.35426694, 0.6019555 , 0.5462863 , 0.49830624,
       0.33782   , 0.1513191 , 0.32407388, 0.46168363, 0.50149894,
       0.41183567, 0.4824324 , 0.5470988 , 0.4686382 , 0.4218773 ,
       0.30310422, 0.6657211 , 0.71908617, 0.3406869 , 0.42012456,
       0.37711495, 0.49952728, 0.340942  , 0.6487066 , 0.3664791 ,
       0.45671022, 0.33055526, 0.44494143, 0.39897966, 0.6989209 ,
       0.43806735, 0.4834198 , 0.44958937, 0.52118397, 0.6196227 ,
       0.37030318, 0.507506  , 0.32592154, 0.48225653, 0.44190446,
       0.24580313, 0.4824325 , 0.24610229, 0.42080426, 0.45009

In [ ]:
from cleanlab.filter import find_label_issues

ordered_label_issues = find_label_issues(
    labels=labels,
    pred_probs=pred_probs,
    return_indices_ranked_by='self_confidence',
)

In [124]:
ood_train_feature_scores

array([0.583817  , 0.5571576 , 0.53148454, 0.12195943, 0.4530235 ,
       0.5697815 , 0.41915447, 0.41873714, 0.30173627, 0.3359073 ,
       0.36623478, 0.3200014 , 0.5039719 , 0.29378122, 0.48308095,
       0.53739303, 0.528277  , 0.343223  , 0.4126202 , 0.38446197,
       0.6674264 , 0.29366848, 0.50010955, 0.6280158 , 0.4387273 ,
       0.44971368, 0.5746294 , 0.24315289, 0.45362303, 0.38513872,
       0.6210135 , 0.35426694, 0.6019555 , 0.5462863 , 0.49830624,
       0.33782   , 0.1513191 , 0.32407388, 0.46168363, 0.50149894,
       0.41183567, 0.4824324 , 0.5470988 , 0.4686382 , 0.4218773 ,
       0.30310422, 0.6657211 , 0.71908617, 0.3406869 , 0.42012456,
       0.37711495, 0.49952728, 0.340942  , 0.6487066 , 0.3664791 ,
       0.45671022, 0.33055526, 0.44494143, 0.39897966, 0.6989209 ,
       0.43806735, 0.4834198 , 0.44958937, 0.52118397, 0.6196227 ,
       0.37030318, 0.507506  , 0.32592154, 0.48225653, 0.44190446,
       0.24580313, 0.4824325 , 0.24610229, 0.42080426, 0.45009

**Obtain the indices of outliers in both the training and test datasets with cleanlab**

In [10]:
from cleanlab.rank import find_top_issues
top = 100
top_train_ood_features_idxs = find_top_issues(quality_scores=ood_train_feature_scores, 
                                              top=top)
top_test_ood_features_idxs = find_top_issues(quality_scores=ood_test_feature_scores, 
                                              top=top)

In [41]:
train_data_path

'/kaggle/working/output/train.csv'

**Remove the outliers from both the training and test datasets, and save the results**

In [42]:
import pandas as pd
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
train_df_new = train_df.drop(top_train_ood_features_idxs, inplace=False)
test_df_new = test_df.drop(top_test_ood_features_idxs, inplace=False)

/tmp/ipykernel_34/2134907574.py:2: DtypeWarning: Columns (145,156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_data_path)
/tmp/ipykernel_34/2134907574.py:3: DtypeWarning: Columns (162) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_data_path)


In [44]:
import os 
# if not os.path.exists("/kaggle/working/output/"):
#     os.mkdir("/kaggle/working/output/")
train_df_new.to_csv(outlier_removed_train_data_path, index=False)
test_df_new.to_csv(outlier_removed_test_data_path, index=False)

**Train the MLP classifier on the datasets, following the removal of outliers**

In [51]:
"""The trained mlp classifier will be save in /kaggle/working/output/mlp.pt 
if you run the following command:
"""
import os
command = f"""
python /kaggle/working/scarf/run_mlp.py \
  --config_dir={config_dir} \
  --output_dir={output_dir}\
  --train_data_path={outlier_removed_train_data_path} \
  --test_data_path={outlier_removed_test_data_path} \
  --batch_size 32 \
  --epochs 1 \
  --lr 0.00003 \
  --model_name={mlp_model_name} \
  --wandb_project_name "test" \
  --wandb_entity "urbancomp" \
  --wandb_key {wandb_key} \
  --hidden_layer 256 128 64 32 16 \
  --dropout 0.1
"""

os.system(command)

wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240605_163621-kevch2mz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run test
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf-MLP
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf-MLP/runs/kevch2mz


Model saved at /kaggle/working/output/mlp.pt
Epoch [1/1] - Train Loss: 1.350, Train Acc: 0.600, Train F1: 0.444, Test Loss: 1.711, Test Acc: 0.349, Test F1: 0.208
Test Accuracy: 0.349, Test F1: 0.208


wandb: \ 0.114 MB of 0.134 MB uploaded
wandb: Run history:
wandb:      test/A1_acc ▁
wandb:      test/A2_acc ▁
wandb:      test/A3_acc ▁
wandb:      test/B1_acc ▁
wandb:      test/B2_acc ▁
wandb:      test/B3_acc ▁
wandb:      test/C1_acc ▁
wandb:      test/C2_acc ▁
wandb:      test/C3_acc ▁
wandb:      test/D1_acc ▁
wandb:      test/D2_acc ▁
wandb:      test/E1_acc ▁
wandb:      test/E2_acc ▁
wandb:       test/F_acc ▁
wandb:       test/G_acc ▁
wandb:    test/test_acc ▁
wandb:     test/test_f1 ▁
wandb:   test/test_loss ▁
wandb:         train/f1 ▁▁▁▁▁▁▂▃▂▂▂▃▄▂▅▅█▆▃▅▅▄▃▇▄▄▆▅▃▆▇▅▆▇▄▆▄▆▆█
wandb:  train/train_acc ▂▄▂▁▂▃▂▅▃▃▂▄▅▂▆▅▆▆▄▄▅▄▃▆▅▄▆▅▃▆▆▅▆▇▄▇▄▇▆█
wandb: train/train_loss ███████▇▇▆▆▆▅▆▄▄▃▄▅▃▁▆▅▂▃▃▂▂▅▂▁▃▃▂▄▃▃▁▁▁
wandb: 
wandb: Run summary:
wandb:      test/A1_acc 0.0
wandb:      test/A2_acc 0.96881
wandb:      test/A3_acc 0.1964
wandb:      test/B1_acc 0.0
wandb:      test/B2_acc 0.0
wandb:      test/B3_acc 0.06188
wandb:      test/C1_acc 0.71779
wandb:      test/C2_acc 0.48316
wandb: 

0